#Milestone 2.2 - Hash generation

Steps done here:

1. Compute spectrogram (STFT magnitude)
2. Compute constellation map (peak picking)
3. Extract peaks (time, frequency, magnitude)
4. Generate **anchor–target hashes** inside configurable **target zones**
5. Store hashes in an **inverted index**: `hash -> [(track_id, time_offset), ...]`
6. Save the database and a small report (hash counts + file sizes)


In [21]:
# Imports
import os
import glob
import time
import pickle
from collections import defaultdict

import numpy as np
import pandas as pd
import librosa
from scipy import ndimage

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')


## Configuration

Edit `BASE_PATH`, `DB_AUDIO_PATH`, and `OUTPUT_*` to match your machine.

- `DB_AUDIO_PATH` should point to the folder containing the database tracks.
- The code supports `.mp3`, `.wav`, `.m4a`.


In [22]:
# Paths
BASE_PATH = r"C:/Development/MusicIR"
DB_AUDIO_PATH = os.path.join(BASE_PATH, "raw_30s_audio-26")

# Output folder for Task 1
OUTPUT_DIR = os.path.join(BASE_PATH, "task1_hash_db")
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_DB_FILE = os.path.join(OUTPUT_DIR, "hash_db.pkl")
OUTPUT_REPORT_CSV = os.path.join(OUTPUT_DIR, "hash_db_report.csv")

print("DB_AUDIO_PATH:", DB_AUDIO_PATH)
print("OUTPUT_DB_FILE:", OUTPUT_DB_FILE)
print("OUTPUT_REPORT_CSV:", OUTPUT_REPORT_CSV)


DB_AUDIO_PATH: C:/Development/MusicIR\raw_30s_audio-26
OUTPUT_DB_FILE: C:/Development/MusicIR\task1_hash_db\hash_db.pkl
OUTPUT_REPORT_CSV: C:/Development/MusicIR\task1_hash_db\hash_db_report.csv


## Parameters

- **Peak picking**: we reuse the best-performing configuration from MS1: `dist_freq=12`, `dist_time=6`, `thresh=0.01`.
- **Target zones**: four configurations using two frequency ranges × two time ranges (in frames).

> Note: time windows are expressed in **frames**, because hashes are based on the time-frame index from the STFT.


In [23]:
# --- STFT parameters ---
Fs = 22050
N = 2048
H = 1024

# --- Peak picking parameters ---
PEAK_DIST_FREQ = 12
PEAK_DIST_TIME = 6
PEAK_THRESH = 0.01

# restrict bins to fit into 10-bit frequency (0..1023) cleanly
# N=2048 produces 1025 bins (0..1024). Setting BIN_MAX=1024 drops the Nyquist bin.
BIN_MAX = 1024

# --- Target zones (two freq ranges × two time ranges) ---
TARGET_ZONES = [
    {"name": "Fsmall_Tshort", "df": 50,  "dt_min": 1, "dt_max": 20},
    {"name": "Fsmall_Tlong",  "df": 50,  "dt_min": 1, "dt_max": 60},
    {"name": "Flarge_Tshort", "df": 150, "dt_min": 1, "dt_max": 20},
    {"name": "Flarge_Tlong",  "df": 150, "dt_min": 1, "dt_max": 60},
]

# limits number of target peaks per anchor (makes index size smaller)
TOP_K_TARGETS = 5  # can be set to None to disable

# Duration handling:
# - Use None for full track (needed in MS2.2 Task 3 scaling)
LOAD_DURATION = 30.0

sec_per_frame = H / Fs
print(f"sec_per_frame = {sec_per_frame:.5f} s")
print("Target zones:")
for z in TARGET_ZONES:
    print(" ", z, f"(~dt_max {z['dt_max']*sec_per_frame:.2f}s)")


sec_per_frame = 0.04644 s
Target zones:
  {'name': 'Fsmall_Tshort', 'df': 50, 'dt_min': 1, 'dt_max': 20} (~dt_max 0.93s)
  {'name': 'Fsmall_Tlong', 'df': 50, 'dt_min': 1, 'dt_max': 60} (~dt_max 2.79s)
  {'name': 'Flarge_Tshort', 'df': 150, 'dt_min': 1, 'dt_max': 20} (~dt_max 0.93s)
  {'name': 'Flarge_Tlong', 'df': 150, 'dt_min': 1, 'dt_max': 60} (~dt_max 2.79s)


## Core functions (spectrogram + constellation map)

This is your MS1 implementation with two important fixes:

1. `frame_max` now defaults to the **time** dimension (`X.shape[1]`), not the freq dimension.
2. `duration` is parameterized via `LOAD_DURATION`.


In [24]:
def compute_spectrogram(fn_wav, Fs=22050, N=2048, H=1024, duration=30.0, bin_max=None, frame_max=None):
    """Computes the magnitude spectrogram of an audio file.

    Args:
        fn_wav (str): Path to the audio file
        Fs (int): Sampling rate in Hz
        N (int): FFT/window size
        H (int): Hop size
        duration (float|None): Seconds to load; None loads full audio
        bin_max (int|None): Max frequency bin (exclusive)
        frame_max (int|None): Max time frame (exclusive)

    Returns:
        Y (np.ndarray): Magnitude spectrogram (freq x time)
    """
    x, Fs = librosa.load(fn_wav, sr=Fs, duration=duration)

    X = librosa.stft(
        x,
        n_fft=N,
        hop_length=H,
        win_length=N,
        window='hann'
    )

    if bin_max is None:
        bin_max = X.shape[0]
    if frame_max is None:
        frame_max = X.shape[1]  # FIX: time dimension

    Y = np.abs(X[:bin_max, :frame_max])
    return Y


def compute_constellation_map(Y, dist_freq=7, dist_time=7, thresh=0.01):
    """Compute constellation map via max-filter peak picking.

    Args:
        Y (np.ndarray): Magnitude spectrogram
        dist_freq (int): neighborhood in freq
        dist_time (int): neighborhood in time
        thresh (float): min magnitude threshold

    Returns:
        Cmap (np.ndarray): boolean mask of peaks (same shape as Y)
    """
    result = ndimage.maximum_filter(
        Y,
        size=[2 * dist_freq + 1, 2 * dist_time + 1],
        mode='constant'
    )
    Cmap = np.logical_and(Y == result, result > thresh)
    return Cmap


## Peak extraction

Convert the boolean constellation map to a **time-sorted** list of peaks:

`(t_frame, f_bin, magnitude)`

Time-sorting is essential so we can stop scanning targets once we pass `dt_max`.


In [25]:
def extract_peaks(Y, Cmap):
    """Extract peaks from constellation map as a list of (t, f, mag), sorted by t.

    Args:
        Y (np.ndarray): magnitude spectrogram (freq x time)
        Cmap (np.ndarray): boolean peak mask

    Returns:
        peaks (list[tuple[int,int,float]]): (t, f, mag) sorted by t
    """
    f_idx, t_idx = np.where(Cmap)
    mags = Y[f_idx, t_idx]
    peaks = list(zip(t_idx.tolist(), f_idx.tolist(), mags.tolist()))
    peaks.sort(key=lambda x: x[0])
    return peaks


## 32-bit hash packing

We pack `(f_anchor, f_target, Δt)` into a 32-bit unsigned integer:

- 10 bits: f_anchor (0..1023)
- 10 bits: f_target (0..1023)
- 12 bits: dt (0..4095)

This fits the assignment requirement of a **32-bit hash**.


In [27]:
BITS_F = 10
BITS_DT = 12
MAX_F = (1 << BITS_F) - 1       # 1023
MAX_DT = (1 << BITS_DT) - 1     # 4095


def make_hash(f1, f2, dt):
    """Create a 32-bit hash for (f1, f2, dt).

    Returns None if dt is out of range (shouldn't happen with our dt_max).
    """
    # enforce representable ranges
    if f1 > MAX_F: f1 = MAX_F
    if f2 > MAX_F: f2 = MAX_F
    if dt < 0 or dt > MAX_DT:
        return None

    # (f1 << 22) | (f2 << 12) | dt
    return (int(f1) << (BITS_F + BITS_DT)) | (int(f2) << BITS_DT) | int(dt)


def pack_posting(track_id, t_anchor):
    """Pack (track_id, t_anchor) into one 32-bit integer (16 bits each).

    If you have more than 65535 tracks or time frames, switch to 64-bit packing.
    """
    return (int(track_id) << 16) | (int(t_anchor) & 0xFFFF)


def unpack_posting(p):
    track_id = int(p) >> 16
    t_anchor = int(p) & 0xFFFF
    return track_id, t_anchor


## Hash generation (anchor → targets inside target zone)

For each anchor peak, we consider peaks in the time window `[dt_min, dt_max]` and within a frequency band `±df`.

To keep the index size manageable, we optionally keep only the **TOP_K_TARGETS** strongest targets per anchor.


In [28]:
def generate_hashes_from_peaks(peaks, zone, top_k=None):
    """Generate (hash, anchor_time) pairs for one track and one target zone.

    Args:
        peaks: list of (t, f, mag) sorted by t
        zone: dict with keys {df, dt_min, dt_max}
        top_k: keep only K strongest targets per anchor (by target mag). None disables.

    Returns:
        pairs: list of (hash_value, t_anchor)
    """
    df = zone["df"]
    dt_min = zone["dt_min"]
    dt_max = zone["dt_max"]

    out = []
    n = len(peaks)

    for i in range(n):
        t1, f1, m1 = peaks[i]

        candidates = []
        for j in range(i + 1, n):
            t2, f2, m2 = peaks[j]
            dt = t2 - t1

            if dt < dt_min:
                continue
            if dt > dt_max:
                break  # peaks are time-sorted

            if abs(f2 - f1) <= df:
                candidates.append((f2, dt, m2))

        if top_k is not None and len(candidates) > top_k:
            candidates.sort(key=lambda x: x[2], reverse=True)
            candidates = candidates[:top_k]

        for f2, dt, m2 in candidates:
            h = make_hash(f1, f2, dt)
            if h is not None:
                out.append((h, t1))

    return out


## Build hash database (inverted index)

We build and store, for each target zone:

- `index[zone_name][hash] = [posting, posting, ...]`

where `posting = pack_posting(track_id, t_anchor)`.

We also store metadata required by the team for matching later (Fs/N/H, peak params, zone params, file list).


In [30]:
def list_audio_files(folder):
    exts = ('*.mp3', '*.wav', '*.m4a')
    files = []
    for ext in exts:
        files.extend(glob.glob(os.path.join(folder, ext)))
    files = sorted(files)
    return files


def build_hash_database(db_audio_path, output_db_file, zones,
                        Fs=22050, N=2048, H=1024, bin_max=None,
                        peak_dist_freq=12, peak_dist_time=6, peak_thresh=0.01,
                        duration=30.0, top_k_targets=5):
    """Build a hash DB with multiple target zones.

    Returns:
        db (dict): database structure
        report_rows (list[dict]): per-zone stats
    """

    db_files = list_audio_files(db_audio_path)
    if not db_files:
        raise FileNotFoundError(f"No audio files found in: {db_audio_path}")

    # index per zone: zone_name -> dict(hash -> list(postings))
    index_by_zone = {z["name"]: defaultdict(list) for z in zones}

    # stats
    per_zone_hash_counts = {z["name"]: 0 for z in zones}
    per_track_hash_counts = {z["name"]: [] for z in zones}

    t0 = time.time()

    tracks = {}  # track_id -> filename

    for track_id, fpath in enumerate(tqdm(db_files, desc="Indexing DB (Task 1)")):
        fname = os.path.basename(fpath)
        tracks[track_id] = fname

        Y = compute_spectrogram(
            fpath, Fs=Fs, N=N, H=H,
            duration=duration,
            bin_max=bin_max
        )

        Cmap = compute_constellation_map(
            Y,
            dist_freq=peak_dist_freq,
            dist_time=peak_dist_time,
            thresh=peak_thresh
        )

        peaks = extract_peaks(Y, Cmap)

        # generate and store hashes for each zone
        for z in zones:
            zname = z["name"]
            pairs = generate_hashes_from_peaks(peaks, z, top_k=top_k_targets)

            for h, t_anchor in pairs:
                index_by_zone[zname][h].append(pack_posting(track_id, t_anchor))

            per_zone_hash_counts[zname] += len(pairs)
            per_track_hash_counts[zname].append(len(pairs))

    elapsed = time.time() - t0

    db = {
        "meta": {
            "Fs": Fs, "N": N, "H": H,
            "duration": duration,
            "bin_max": bin_max,
            "peak_params": {
                "dist_freq": peak_dist_freq,
                "dist_time": peak_dist_time,
                "thresh": peak_thresh
            },
            "zones": zones,
            "top_k_targets": top_k_targets,
            "created_at_unix": time.time(),
            "elapsed_seconds": elapsed
        },
        "tracks": tracks,
        "index": {zname: dict(index_by_zone[zname]) for zname in index_by_zone}
    }

    with open(output_db_file, 'wb') as f:
        pickle.dump(db, f, protocol=pickle.HIGHEST_PROTOCOL)

    # Build report rows
    report_rows = []
    for z in zones:
        zname = z["name"]
        total_hashes = per_zone_hash_counts[zname]
        avg_hashes = float(np.mean(per_track_hash_counts[zname])) if per_track_hash_counts[zname] else 0.0
        report_rows.append({
            "zone": zname,
            "df_bins": z["df"],
            "dt_min_frames": z["dt_min"],
            "dt_max_frames": z["dt_max"],
            "total_hashes": int(total_hashes),
            "avg_hashes_per_track": avg_hashes,
            "num_tracks": len(db_files),
            "top_k_targets": top_k_targets,
        })

    return db, report_rows


## Run Task 1: build + save DB

This cell will:

- build the hash DB
- save `hash_db.pkl` into `OUTPUT_DIR`
- write `hash_db_report.csv`
- show a small summary

If `hash_db.pkl` already exists, it will **load** it instead of rebuilding (delete the file if you want to rebuild).


In [31]:
if os.path.exists(OUTPUT_DB_FILE):
    print(f"Found existing DB at: {OUTPUT_DB_FILE}")
    print("Loading it...")
    with open(OUTPUT_DB_FILE, 'rb') as f:
        db = pickle.load(f)


    rows = []
    for z in db["meta"]["zones"]:
        zname = z["name"]
        idx = db["index"][zname]
        total_hashes = sum(len(v) for v in idx.values())
        rows.append({
            "zone": zname,
            "df_bins": z["df"],
            "dt_min_frames": z["dt_min"],
            "dt_max_frames": z["dt_max"],
            "total_hashes": int(total_hashes),
            "num_tracks": len(db["tracks"]),
            "top_k_targets": db["meta"].get("top_k_targets"),
        })

    report_df = pd.DataFrame(rows)
else:
    db, rows = build_hash_database(
        DB_AUDIO_PATH,
        OUTPUT_DB_FILE,
        zones=TARGET_ZONES,
        Fs=Fs, N=N, H=H,
        bin_max=BIN_MAX,
        peak_dist_freq=PEAK_DIST_FREQ,
        peak_dist_time=PEAK_DIST_TIME,
        peak_thresh=PEAK_THRESH,
        duration=LOAD_DURATION,
        top_k_targets=TOP_K_TARGETS
    )

    report_df = pd.DataFrame(rows)


file_size_bytes = os.path.getsize(OUTPUT_DB_FILE)
report_df["db_file_size_bytes"] = file_size_bytes
report_df.to_csv(OUTPUT_REPORT_CSV, index=False)

print("Task 1 Summary")
print("Saved:", OUTPUT_DB_FILE)
print("DB size:", file_size_bytes, "bytes")
print("Saved report:", OUTPUT_REPORT_CSV)

display(report_df)



Indexing DB (Task 1): 100%|██████████| 20/20 [01:00<00:00,  3.01s/it]


Task 1 Summary
Saved: C:/Development/MusicIR\task1_hash_db\hash_db.pkl
DB size: 7619377 bytes
Saved report: C:/Development/MusicIR\task1_hash_db\hash_db_report.csv


,zone,df_bins,dt_min_frames,dt_max_frames,total_hashes,avg_hashes_per_track,num_tracks,top_k_targets,db_file_size_bytes
0,Fsmall_Tshort,50,1,20,134524,6726.20,20,5,7619377
1,Fsmall_Tlong,50,1,60,160971,8048.55,20,5,7619377
2,Flarge_Tshort,150,1,20,161834,8091.70,20,5,7619377
3,Flarge_Tlong,150,1,60,163209,8160.45,20,5,7619377


## Statistics

- number of tracks
- number of unique hashes per zone
- length of some posting lists


In [33]:
print("Tracks:", len(db["tracks"]))

for z in db["meta"]["zones"]:
    zname = z["name"]
    idx = db["index"][zname]
    unique_hashes = len(idx)
    total_postings = sum(len(v) for v in idx.values())
    avg_postings_per_hash = total_postings / unique_hashes if unique_hashes else 0

    print(f"Zone: {zname}")
    print("  unique hashes:", unique_hashes)
    print("  total postings:", total_postings)
    print("  avg postings/hash:", avg_postings_per_hash)

    # show a few example posting-list lengths
    if unique_hashes:
        sample_keys = list(idx.keys())[:5]
        print("  sample posting list lengths:", [len(idx[k]) for k in sample_keys])


Tracks: 20
Zone: Fsmall_Tshort
  unique hashes: 105660
  total postings: 134524
  avg postings/hash: 1.2731781184932804
  sample posting list lengths: [2, 1, 1, 1, 1]
Zone: Fsmall_Tlong
  unique hashes: 133589
  total postings: 160971
  avg postings/hash: 1.2049719662547067
  sample posting list lengths: [1, 1, 1, 1, 1]
Zone: Flarge_Tshort
  unique hashes: 130838
  total postings: 161834
  avg postings/hash: 1.2369036518442653
  sample posting list lengths: [1, 1, 1, 1, 1]
Zone: Flarge_Tlong
  unique hashes: 136604
  total postings: 163209
  avg postings/hash: 1.194760036309332
  sample posting list lengths: [1, 1, 1, 1, 1]
